----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - Pricing stream - How to use streaming events
This Notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) either directly from the Refinitiv Data Platform or via Refinitiv Workspace or CodeBook. The example will show how to create a Pricing stream object with registered event handlers so that your application is notified when new data is coming in.

Using Pricing streams this way allows your application to be updated in real-time when data changes on the market. With this event-driven mode, your application can still benefit from the Pricing stream data cache and use the get_snapshot function to pull out real-time snapshots as Pandas DataFrames.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help.

## Introduction to streaming events

Using a Pricing stream object with events requires you to define event handlers that are called by a background execution thread when new events are received for the instruments you requested. There are 4 different types of event handlers you can optionnaly define depending on the type to events your are interrested in.  

### The 4 event types and their related event handlers:
 - **Refresh events:** Refresh events happen when all fields of one the requested instruments are received. This complete list of fields is sometimes called the 'image' of the instrument. This image that comes with Refresh messages can be later updated by subsequent Update events. When several Refresh events are received for the same instrument, the fields transported by the latest Refresh are considered as the new image. Fields received in previous Refresh events or Update events must be discarded. Pricing stream objects automatically manage this logic for their internal cache, meaning that when you call get_snapshot you always get the latest and relevant field values for the requested instruments. 
 
 Refresh event handlers take 3 parameters: 
  - The Pricing stream object that received the event
  - The name of the concerned instrument
  - The fields and values of the Image
 
 
 - **Update events:** Update events are received when fields of a requested instrument change. Update events only contain the fields and values that changed. When the application receives an Update it must update its internal representation of the instrument (if any) accordingly. Pricing stream objects automatically manage this logic for their internal cache, meaning that when you call get_snapshot you always get the latest values of the requested instruments. 
 
 Update event handlers take 3 parameters 
  - The Pricing stream object that received the event
  - The name of the concerned instrument
  - The updated fields with their new values
 
 
 - **Status events:** Status events are received when the status of one of the requested instruments changes.
 
 Status event handlers take 3 parameters 
  - The Pricing stream object that received the event
  - The name of the concerned instrument
  - The new status of the instrument
 
 
 - **Complete events:** A Complete event is received once all the requested instruments received either a Refresh or a Status event. The Complete event indicates that the Pricing stream object is complete and that it's internal cache contains the full data set (instruments and fields) that were requested.     

 Complete event handlers take one parameter
  - The Pricing stream object that is complete
 
**Side note:** As Refresh events and Update events use handlers with the same signature, the same handler can be used for these 2 event types if you do not need to distinguish them 
 
### Typical events flow

As an example, if you use a Pricing stream object with event for the following instruments and fields: 
 - Instruments: 'CAD=','GBP=', 'JPY=', 'JUNK'
 - Fields: 'CF_BID','CF_ASK','OPEN_PRC', 'CF_HIGH','CF_LOW', 'CF_CLOSE'

You may receive a flow of events like this one:
 1. **Refresh** event for GBP=
 1. **Refresh** event for CAD=
 1. **Status** event for JUNK
 1. **Refresh** event for JPY=
 1. **Complete** event => Indicating that data (or status) has been received for all requested instruments
 1. **Update** event for JPY=
 1. **Update** event for CAD=
 1. **Refresh** event for CAD=
 1. **Update** event for GBP=
 1. **Update** event for CAD=
 1. **Update** event for JPY=
 1. ...


## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [2]:
import refinitiv.data as rd
from refinitiv.data.content import pricing
import datetime

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [3]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fb58528ba90 {name='workspace'}>

## Retrieve data

### Define callbacks to capture incoming events

The following function will be used to capture Refresh events. It displays the name of the refreshed instrument and its full image (complete list of requested fields).

In [4]:
def display_refreshed_fields(pricing_stream, instrument_name, fields):
    current_time = datetime.datetime.now().time()
    print(current_time, "- Refresh received for", instrument_name, ":", fields)    

The following function will be used to capture Update events. It displays the name of the updated instrument and the updated fields.

In [5]:
def display_updated_fields(pricing_stream, instrument_name, fields):
    current_time = datetime.datetime.now().time()
    print(current_time, "- Update received for", instrument_name, ":", fields)    

The following function will be used to capture Status events. It displays the name of the instrument and the received status.

In [6]:
def display_status(pricing_stream, instrument_name, status):
    current_time = datetime.datetime.now().time()
    print(current_time, "- Status received for", instrument_name, ":", status)    

The following function will be used to capture Complete events. It calls get_snapshot to pull out the memory cache of the Pricing stream as a Pandas DataFrame and displays it. As a result, the latest values of all requested fields and instruments are displayed in a table. 

In [7]:
def display_complete_snapshot(pricing_stream):
    current_time = datetime.datetime.now().time()
    print(current_time, "- Pricing stream is complete. Full snapshot:")
    display(pricing_stream.get_snapshot())

### Create a Pricing stream and register event callbacks

In [8]:
stream = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()


stream.on_refresh(display_refreshed_fields)
stream.on_update(display_updated_fields)
stream.on_status(display_status)
stream.on_complete(display_complete_snapshot)

<refinitiv.data.content.pricing.Stream object at 0x7fb58241de90 {name='['EUR=', 'GBP=', 'JPY=', 'CAD=']'}>

### Open the stream

In [9]:
stream.open()

14:10:04.230524 - Refresh received for EUR= : {'BID': 1.1332, 'ASK': 1.1333}
14:10:04.294298 - Refresh received for GBP= : {'BID': 1.3353, 'ASK': 1.3355}
14:10:04.296576 - Refresh received for JPY= : {'BID': 114.14, 'ASK': 114.15}
14:10:04.319355 - Refresh received for CAD= : {'BID': 1.2779, 'ASK': 1.2783}
14:10:04.319584 - Pricing stream is complete. Full snapshot:


,Instrument,ASK,BID
0,EUR=,1.1333,1.1332
1,GBP=,1.3355,1.3353
2,JPY=,114.15,114.14
3,CAD=,1.2783,1.2779


<OpenState.Opened: 1>

14:10:04.358514 - Update received for GBP= : {'BID': 1.3352, 'ASK': 1.3356}
14:10:04.359312 - Update received for JPY= : {'BID': 114.13, 'ASK': 114.16}
14:10:04.747091 - Update received for CAD= : {'BID': 1.2782, 'ASK': 1.2786}
14:10:04.778022 - Update received for EUR= : {'BID': 1.1331, 'ASK': 1.1335}
14:10:04.914487 - Update received for GBP= : {'BID': 1.3355, 'ASK': 1.3356}
14:10:05.259574 - Update received for GBP= : {'BID': 1.3353, 'ASK': 1.3354}
14:10:05.267195 - Update received for JPY= : {'BID': 114.13, 'ASK': 114.16}
14:10:05.918976 - Update received for CAD= : {'BID': 1.2779, 'ASK': 1.2783}
14:10:05.965406 - Update received for GBP= : {'BID': 1.3353, 'ASK': 1.3356}
14:10:05.966023 - Update received for EUR= : {'BID': 1.1332, 'ASK': 1.1334}
14:10:06.238758 - Update received for EUR= : {'BID': 1.1331, 'ASK': 1.1335}
14:10:06.289881 - Update received for JPY= : {'BID': 114.14, 'ASK': 114.15}


### Close the stream

In [10]:
stream.close()

14:10:06.528086 - Update received for EUR= : {'BID': 1.133, 'ASK': 1.1334}


<OpenState.Closed: 3>

## Close the session

In [11]:
rd.close_session()